# Accuracy Assessments of crop-mask results

Inputs will be:

1. `predicted.tif` : a binary classification of crop/no-crop output by the ML script.

2. `cropland_prelim_validation.shp` : a shapefile containing crop/no-crop points to serve as the "ground-truth" dataset

Output will be:
1. A confusion matrix or something akin to that, % accuracy for omission and commission

2. Some kind of spatial comparison? Maybe this can just be a qualitative visual assessment

In [ ]:
import sys
import os
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from sklearn.metrics import accuracy_score, jaccard_score, confusion_matrix
from sklearn.utils.multiclass import unique_labels

sys.path.append('../Scripts')
from deafrica_plotting import rgb
from deafrica_datahandling import mostcommon_crs, load_ard
from deafrica_dask import create_local_dask_cluster
from deafrica_spatialtools import xr_rasterize

### Load the datasets

In [ ]:
ground_truth = gpd.read_file('data/training_data/cropland_prelim_validation.shp').to_crs('EPSG:6933')
ground_truth.head()

In [ ]:
prediction = xr.open_rasterio('results/predicted.tif').squeeze()
# attrs = prediction.attrs
# prediction = prediction.astype(bool)
# prediction.attrs = attrs

### Convert ground truth data to xarray

In [ ]:
ground_truth = xr_rasterize(ground_truth, prediction, attribute_col='class')
# ground_truth = ground_truth.astype(bool)
# ground_truth.attrs = attrs

### Jaccard Similarity index and overall accuracy

In [ ]:
jss = jaccard_score(ground_truth, prediction, average='binary')
ac = accuracy_score(ground_truth, prediction, normalize=True)

In [ ]:
print("The Normalised Jaccard Similarity Score is: "+ str(round(jss, 2)))
print("The Normalised Accuracy Score is: "+ str(round(ac, 2)))

### Create a confusion matrix

In [ ]:
np.min(ground_truth.values)

In [ ]:
tn, fp, fn, tp = confusion_matrix(ground_truth.values,
                                  prediction.values).ravel()

#.argmax(axis=1)

In [ ]:
print(tn, fp, fn, tp)

In [ ]:
def plot_confusion_matrix(y_true,
                          y_pred,
                          classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = list(unique_labels(y_true, y_pred))
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
#     plt.xlim(-0.5, 5.5)
#     plt.ylim(5.5, -0.5)
    
    fig.tight_layout()
    return ax


In [ ]:
plot_confusion_matrix(ground_truth.values.argmax(axis=1),
                      prediction.values.argmax(axis=1),
                      classes=[0, 1],
                      normalize=True,
                      title='Normalized confusion matrix')